In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [2]:
path = "Downloads/Crop Yiled with Soil and Weather.csv"
df = pd.read_csv(path)

In [3]:
df.drop_duplicates()

,Fertilizer,temp,N,P,K,yeild
0,80.000000,28.000000,80.000000,24.000000,20.000000,12.000000
1,77.000000,27.000000,78.000000,23.000000,20.000000,12.000000
2,80.000000,26.000000,80.000000,24.000000,20.000000,12.000000
4,78.000000,27.000000,78.000000,23.000000,19.000000,12.000000
6,75.000000,26.000000,75.000000,22.000000,19.000000,12.000000
...,...,...,...,...,...,...
2591,78.942684,27.014634,76.838312,23.087606,20.044206,10.348503
2592,79.077362,27.153127,76.927657,23.068234,20.155436,10.528122
2593,78.986271,28.108948,77.164352,22.863654,20.989871,10.564205
2594,78.894767,28.141344,76.925149,23.071553,20.873662,10.442511


In [4]:
X = df[['Fertilizer','temp','N','P','K']]
y = df['yeild']
y = y.values.reshape(-1, 1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4, random_state=3)
X_cv, X_test, y_cv, y_test = train_test_split(X_test,y_test,test_size=0.5, random_state=3)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_cv_scaled = scaler.fit_transform(X_cv)
X_test_scaled = scaler.fit_transform(X_test)


In [6]:
def relu(z):
    return np.maximum(0, z)

In [7]:
def relu_derivative(z):
    return (z > 0).astype(float)

In [8]:
def linear(z):
    return z

In [9]:
def weight_bias_initialize():
    np.random.seed(42)
    W1 = np.random.randn(X_train_scaled.shape[1],64)*0.1
    b1 = np.zeros((1, 64))

    W2 = np.random.randn(64,32)*0.1
    b2 = np.zeros((1, 32))

    W3 = np.random.randn(32, 1)*0.1
    b3 = np.zeros((1, 1))

    return (W1,b1,W2,b2,W3,b3)

In [10]:
def compute_final_errors(X, y, X_cv, y_cv, W1, b1, W2, b2, W3, b3):
    z1 = X @ W1 + b1
    a1 = relu(z1)
    z2 = a1 @ W2 + b2
    a2 = relu(z2)
    z3 = a2 @ W3 + b3
    a3 = linear(z3)
    trainError = np.mean((a3 - y) ** 2)

    z1 = X_cv @ W1 + b1
    a1 = relu(z1)
    z2 = a1 @ W2 + b2
    a2 = relu(z2)
    z3 = a2 @ W3 + b3
    a3 = linear(z3)
    cvError = np.mean((a3 - y_cv) ** 2)

    return trainError, cvError

In [11]:
def training_model_lambda(_lambda_arr,alpha,X,y,X_cv,y_cv,epochs,patience=3000, min_delta=1e-4):
    for _lambda in _lambda_arr:
        W1,b1,W2,b2,W3,b3 = weight_bias_initialize()
        best_cv_error = float("inf")
        best_weights = None
        wait = 0
        
        for epoch in range(epochs):
    
            ##########Forward propagation##########
            z1 = X @ W1 + b1 #(2076,5)@(5,64)
            a1 = relu(z1)

            z2 = a1 @ W2 + b2 # (2076,64)@(64,32)
            a2 = relu(z2)

            z3 = a2 @ W3 + b3 #(2076,32)@(32,1)
            a3 = linear(z3)

            error = np.mean((a3 - y)**2 ) + _lambda * (np.sum(np.square(W1)) + np.sum(np.square(W2)) + np.sum(np.square(W3)))
            ##########Backward propagation#############

            dz3 = a3 - y
            dW3 = a2.T @ dz3 + _lambda * W3
            db3 = np.sum(dz3,axis=0,keepdims = True)

            da2 = dz3 @ W3.T         
            dz2 = da2 * relu_derivative(z2)  
            dW2 = a1.T @ dz2 + _lambda * W2        
            db2 = np.sum(dz2, axis=0, keepdims=True)

            da1 = dz2 @ W2.T         
            dz1 = da1 * relu_derivative(z1)  
            dW1 = X.T @ dz1 + _lambda * W1        
            db1 = np.sum(dz1, axis=0, keepdims=True)


            ##########Update Weights####################
            W1 = W1 - alpha*dW1
            b1 = b1 - alpha*db1

            W2 = W2 - alpha*dW2
            b2 = b2 - alpha*db2

            W3 = W3 - alpha*dW3
            b3 = b3 - alpha*db3

            ##########Print error every 1000 Epochs#############
            if epoch % 1000 == 0:
                trainError,cvError = compute_final_errors(X, y, X_cv, y_cv, W1, b1, W2, b2, W3, b3)
                print(f"Epoch {epoch}, Train Error: {trainError:.4f}, Validation Error: {cvError:.4f}")

                 ############# Early stopping logic###############
                if cvError + min_delta < best_cv_error:
                    best_cv_error = cvError
                    best_weights = (W1.copy(), b1.copy(), W2.copy(), b2.copy(), W3.copy(), b3.copy())
                    wait = 0
                else:
                    wait += 1000

                if wait >= patience:
                    print(f"Early stopping at epoch {epoch} for lambda {_lambda:.5f}. Best validation error: {best_cv_error:.4f}")
                    break
        if best_weights:
            W1, b1, W2, b2, W3, b3 = best_weights
            
        trainError,cvError = compute_final_errors(X, y, X_cv, y_cv, W1, b1, W2, b2, W3, b3)
        print(f"\nlamdba:{_lambda:.5f}")
        print(f"Train Error: {trainError:.4f}")
        print(f"Cross validation Error: {cvError:.4f} \n")
        print("***"*15,"\n")

In [12]:
_lambda_arr = np.array([0.00001,0.0001,0.001,0.01,0.1])
alpha = 0.0001
training_model_lambda(_lambda_arr,alpha,X_train_scaled,y_train,X_cv_scaled,y_cv,10000)
#lambda just right = 0.0001

Epoch 0, Train Error: 48.4751, Validation Error: 45.0883
Epoch 1000, Train Error: 0.3630, Validation Error: 0.6885
Epoch 2000, Train Error: 0.2087, Validation Error: 0.5156
Epoch 3000, Train Error: 0.1527, Validation Error: 0.4313
Epoch 4000, Train Error: 0.1231, Validation Error: 0.3772
Epoch 5000, Train Error: 0.1047, Validation Error: 0.3342
Epoch 6000, Train Error: 0.0932, Validation Error: 0.3059
Epoch 7000, Train Error: 0.0779, Validation Error: 0.2731
Epoch 8000, Train Error: 0.0676, Validation Error: 0.2581
Epoch 9000, Train Error: 0.0584, Validation Error: 0.2479

lamdba:0.00001
Train Error: 0.0584
Cross validation Error: 0.2479 

********************************************* 

Epoch 0, Train Error: 48.4751, Validation Error: 45.0883
Epoch 1000, Train Error: 0.3987, Validation Error: 0.2978
Epoch 2000, Train Error: 0.2318, Validation Error: 0.1882
Epoch 3000, Train Error: 0.1606, Validation Error: 0.1513
Epoch 4000, Train Error: 0.1273, Validation Error: 0.1270
Epoch 5000, Tra

In [13]:
def training_model_alpha(alpha_arr,_lambda,X,y,X_cv,y_cv,epochs,patience=3000, min_delta=1e-4):
    for alpha in alpha_arr:
        W1,b1,W2,b2,W3,b3 = weight_bias_initialize()
        best_cv_error = float("inf")
        best_weights = None
        wait = 0
        for epoch in range(epochs):
    
            ##########Forward propagation##########
            z1 = X @ W1 + b1 #(2076,5)@(5,64)
            a1 = relu(z1)

            z2 = a1 @ W2 + b2 # (2076,64)@(64,32)
            a2 = relu(z2)

            z3 = a2 @ W3 + b3 #(2076,32)@(32,1)
            a3 = linear(z3)

            error = np.mean((a3 - y)**2 ) + _lambda * (np.sum(np.square(W1)) + np.sum(np.square(W2)) + np.sum(np.square(W3)))
            ##########Backward propagation#############

            dz3 = a3 - y
            dW3 = a2.T @ dz3 + _lambda * W3
            db3 = np.sum(dz3,axis=0,keepdims = True)

            da2 = dz3 @ W3.T         
            dz2 = da2 * relu_derivative(z2)  
            dW2 = a1.T @ dz2 + _lambda * W2        
            db2 = np.sum(dz2, axis=0, keepdims=True)

            da1 = dz2 @ W2.T         
            dz1 = da1 * relu_derivative(z1)  
            dW1 = X.T @ dz1 + _lambda * W1        
            db1 = np.sum(dz1, axis=0, keepdims=True)


            ##########Update Weights####################
            W1 = W1 - alpha*dW1
            b1 = b1 - alpha*db1

            W2 = W2 - alpha*dW2
            b2 = b2 - alpha*db2

            W3 = W3 - alpha*dW3
            b3 = b3 - alpha*db3

            ##########Print error every 1000 Epochs#############
            if epoch % 1000 == 0:
                trainError,cvError = compute_final_errors(X, y, X_cv, y_cv, W1, b1, W2, b2, W3, b3)
                print(f"Epoch {epoch}, Train Error: {trainError:.4f}, Validation Error: {cvError:.4f}")
            ############# Early stopping logic###############
                if cvError + min_delta < best_cv_error:
                    best_cv_error = cvError
                    best_weights = (W1.copy(), b1.copy(), W2.copy(), b2.copy(), W3.copy(), b3.copy())
                    wait = 0
                else:
                    wait += 1000

                if wait >= patience:
                    print(f"Early stopping at epoch {epoch} for lambda {_lambda:.5f}. Best validation error: {best_cv_error:.4f}")
                    break
                    
        if best_weights:
            W1, b1, W2, b2, W3, b3 = best_weights
        
        trainError,cvError = compute_final_errors(X, y, X_cv, y_cv, W1, b1, W2, b2, W3, b3)
        print(f"\nalpha:{alpha:.5f}")
        print(f"Train Error: {trainError:.4f}")
        print(f"Cross validation Error: {cvError:.4f} \n")
        print("***"*15,"\n")
    return (W1,b1,W2,b2,W3,b3)

In [14]:
alpha_arr = np.array([0.0001,0.00003,0.00001])
_lambda = 0.0001
training_model_alpha(alpha_arr,_lambda,X_train_scaled,y_train,X_cv_scaled,y_cv,10000)

Epoch 0, Train Error: 48.4751, Validation Error: 45.0883
Epoch 1000, Train Error: 0.3987, Validation Error: 0.2978
Epoch 2000, Train Error: 0.2318, Validation Error: 0.1882
Epoch 3000, Train Error: 0.1606, Validation Error: 0.1513
Epoch 4000, Train Error: 0.1273, Validation Error: 0.1270
Epoch 5000, Train Error: 0.1086, Validation Error: 0.1158
Epoch 6000, Train Error: 0.0991, Validation Error: 0.1045
Epoch 7000, Train Error: 0.0803, Validation Error: 0.1004
Epoch 8000, Train Error: 0.0681, Validation Error: 0.1006
Epoch 9000, Train Error: 0.0601, Validation Error: 0.1017

alpha:0.00010
Train Error: 0.0803
Cross validation Error: 0.1004 

********************************************* 

Epoch 0, Train Error: 68.0569, Validation Error: 64.0369
Epoch 1000, Train Error: 0.2760, Validation Error: 0.5121
Epoch 2000, Train Error: 0.2096, Validation Error: 0.1879
Epoch 3000, Train Error: 0.1728, Validation Error: 0.2849
Epoch 4000, Train Error: 0.1641, Validation Error: 0.1667
Epoch 5000, Trai

(array([[ 1.26879633e-02,  1.07624682e-03,  6.80841481e-02,
          2.06740517e-01, -2.73254002e-01,  2.45845239e-04,
          1.64487870e-01,  1.59251852e-01, -1.58955596e-02,
          1.26752742e-02, -7.87640038e-02, -2.10083968e-02,
         -1.28422044e-02, -2.39657823e-01, -2.88973172e-01,
         -2.92262059e-01, -2.04580756e-01, -7.79932065e-02,
         -7.60773280e-02, -5.14711785e-01,  8.60625081e-02,
         -2.12637887e-01, -1.12991010e-02, -1.74834163e-01,
         -1.60711756e-01, -9.81031749e-03, -1.01760912e-01,
          1.29924409e-02, -6.76409012e-02, -2.34972305e-01,
         -7.88113430e-05,  1.77941879e-01,  2.91265622e-01,
         -1.62623412e-01,  5.40420052e-02, -1.08615329e-01,
         -3.61802332e-01, -1.48216843e-01, -1.43981074e-01,
          1.86295364e-02,  5.49200011e-02, -2.94889636e-02,
          3.13521300e-02, -9.78677716e-02, -1.83329437e-01,
         -6.71239207e-02,  4.08496935e-02,  4.38400980e-02,
          7.12350685e-02, -2.83313048e-0

In [15]:
alpha_arr = np.array([0.0001])
_lambda = 0.0001
W1,b1,W2,b2,W3,b3 = training_model_alpha(alpha_arr,_lambda,X_train_scaled,y_train,X_cv_scaled,y_cv,10000)

Epoch 0, Train Error: 48.4751, Validation Error: 45.0883
Epoch 1000, Train Error: 0.3987, Validation Error: 0.2978
Epoch 2000, Train Error: 0.2318, Validation Error: 0.1882
Epoch 3000, Train Error: 0.1606, Validation Error: 0.1513
Epoch 4000, Train Error: 0.1273, Validation Error: 0.1270
Epoch 5000, Train Error: 0.1086, Validation Error: 0.1158
Epoch 6000, Train Error: 0.0991, Validation Error: 0.1045
Epoch 7000, Train Error: 0.0803, Validation Error: 0.1004
Epoch 8000, Train Error: 0.0681, Validation Error: 0.1006
Epoch 9000, Train Error: 0.0601, Validation Error: 0.1017

alpha:0.00010
Train Error: 0.0803
Cross validation Error: 0.1004 

********************************************* 



In [18]:
def my_neuron_network(X,W1,b1,W2,b2,W3,b3):
    z1 = X @ W1 + b1 #(2076,5)@(5,64)
    a1 = relu(z1)

    z2 = a1 @ W2 + b2 # (2076,64)@(64,32)
    a2 = relu(z2)

    z3 = a2 @ W3 + b3 #(2076,32)@(32,1)
    a3 = linear(z3)
    return a3

In [ ]:
y_predict = my_neuron_network(X_test_scaled,W1,b1,W2,b2,W3,b3)

MSE_test =  np.mean((y_predict - y_test)**2)
r2_test = r2_score(y_test, y_predict)

print(f"MSE score:{MSE_test:.4f},r2 score:{r2_test:.3f}")